<div class="list-group" id="list-tab" role="tablist">
<h2 class="list-group-item list-group-item-action active" data-toggle="list" style='background:orange; border:0; color:white' role="tab" aria-controls="home"><center>NBME</center></h2>

<center><img src = "https://media.istockphoto.com/vectors/people-wait-in-hospital-hall-interior-vector-illustration-cartoon-vector-id1256189055?k=20&m=1256189055&s=170667a&w=0&h=H4XhK870bkOst0_YcAooxxZvga_Jq_dG7y-xXEQDU_s=" width = "577" height = "299"/></center> 

Hi,
I am writing this notebook for noobs like me who know nothing. 

I am going to explain you how to tackle this problem using bert and you can use whatever you like by just changing Model and tokenizer and 2-3 other parameters. Even if you Know nothing about key phrase extraction don't worry  read till end and you can tackle this problem. I know its hard in start you may dont have teammate dont know what to do with data not understand evaluation metrics, it happens to everyone in start but you should never give up. one more thing forking others notebook and submitting that notebook will not going to help you. Yes there is nothing wrong in reading others kernels.But you must read it from start to end understand what approach they used appreciate their work by upvoting kernel take inspiration from others and start writing your own code.

<font color='red'>If you find my kernel useful and my efforts appreciable, Please Upvote it , it motivates me to write more Quality content</font>

<div class="list-group" id="list-tab" role="tablist">
<h2 class="list-group-item list-group-item-action active" data-toggle="list" style='background:orange; border:0; color:white' role="tab" aria-controls="home"><center>Contents</center></h2>

1. [About Competition](#about-competition) 
2. [Introduction](#introduction)  
3. [Evaluation Metric](#evaluation-metric)  
4. [Data Cleaning](#data-cleaning)  
5. [Preprocessing](#preprocessing) 
6. [Config](#config)   
7. [Utils](#utils)  
8. [Dataset](#dataset)   
9. [Model](#model)  
10. [Engine](#engine)  
11. [Train](#train)

<div class="list-group" id="list-tab" role="tablist">
<h2 class="list-group-item list-group-item-action active" data-toggle="list" style='background:orange; border:0; color:white' role="tab" aria-controls="home"><center>About Competition</center></h2>

When we visit a doctor,how they interpret our symptoms can determine whether our diagnosis is accurate. But before they are licensed, physician have had a lot of practice writing patient notes, patients history, physical exam findings, and their complaints. Learning and assessing the skill of writing patient notes requires feedback from other doctors a time-intensive process that could be improved with the addition of machine learning.

<div class="list-group" id="list-tab" role="tablist">
<h2 class="list-group-item list-group-item-action active" data-toggle="list" style='background:orange; border:0; color:white' role="tab" aria-controls="home"><center>Introduction</center></h2>

I used bert-base-uncased pretrained model with pytorch.


What we have?

We have 5 .csv files.
1. ```features.csv```
2. ```patients_notes.csv```
3. ```sample_submission.csv```
4. ```test.csv```
5. ```train.csv```

Lets take a look at them one by one.

In [ ]:
import os
import re
import gc
import copy
import pickle
import random
from tqdm import tqdm


import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
import plotly.figure_factory as ff
from sklearn import model_selection

import torch 
import torch.nn as nn
from torch.optim import AdamW
from torch.nn import Sigmoid
from torch.utils.data import DataLoader

import tokenizers
import transformers
from transformers import get_linear_schedule_with_warmup

In [ ]:
train_df = pd.read_csv("../input/nbme-score-clinical-patient-notes/train.csv")
pn_notes_df = pd.read_csv("../input/nbme-score-clinical-patient-notes/patient_notes.csv")
feat_df = pd.read_csv("../input/nbme-score-clinical-patient-notes/features.csv")

print(train_df.shape)
display(train_df.head())

In [ ]:
print(feat_df.shape)
display(feat_df.head())

In [ ]:
print(pn_notes_df.shape)
display(pn_notes_df.head())

we have 6 columns in ```train.csv``` ,  3 columns in ```features.csv``` and  3 columns in ```patient_notes.csv```.
You can take look at data description of competition to understand about attributes/columns of dataframes.

In [ ]:
feat_df.feature_num.nunique(),feat_df.case_num.nunique()

There are 143 unique features and 10 unique cases

In [ ]:
cases = [f"case {i}" for i in feat_df.case_num.unique()]
feature_count = [feat_df[feat_df.case_num == i]["feature_num"].nunique() for i in range(10)]
data = pd.DataFrame({"case":cases,"unique_feature_count":feature_count})
fig = px.bar(data,x="case",y="unique_feature_count",width=500,height=400)
fig.show()

You can see in above bar graph how each cases has their unique features.You can take look at EDA kernels in code section there are so many nice kernels for example this one
* [https://www.kaggle.com/odins0n/nbme-detailed-eda](https://www.kaggle.com/odins0n/nbme-detailed-eda)

If you visited above notebook please dont forgot to upvote if it helped you.


Let's go back to our dataset.
You can see that there is ```case_num```  , ```feature_num``` columns are in ```train_df``` and ```feat_df``` we will merge them on ```["case_num","feature_num"]``` 

In [ ]:
train_df = train_df.merge(feat_df,how="left",on=["case_num","feature_num"])
train_df.head()

Now merge this ```train_df``` with ```pn_notes_df``` on ```["case_num","pn_num"]```

In [ ]:
train_df = train_df.merge(pn_notes_df,how="left",on=["case_num","pn_num"])
train_df.head()

<div class="list-group" id="list-tab" role="tablist">
<h2 class="list-group-item list-group-item-action active" data-toggle="list" style='background:orange; border:0; color:white' role="tab" aria-controls="home"><center>Evaluation Metric</center></h2>

Evaluation metric is micro_average_F1 score
and it's like we have two targets one is predicted and another one is original Then in our case targets is location span 
example if we have 

              predicted location ["333 342"]
              orig location      ["332 340"]
Then our ground truth is span of orignal location and prediction is span of predicted location
like this 

              ground truth = [333, 334, 335, 336, 337, 338, 339, 340, 341]
              prediction =   [333, 334, 335, 336, 337, 338, 339]
             TP = len([333, 334, 335, 336, 337, 338, 339]) = 7
             FP = len([]) = 0
             FN = len([340, 341]) = 2
             
             NOW SUM  TP FP FN individualy and calculate final F1 score 
    

<div class="list-group" id="list-tab" role="tablist">
<h2 class="list-group-item list-group-item-action active" data-toggle="list" style='background:orange; border:0; color:white' role="tab" aria-controls="home"><center>Data Cleaning</center></h2>

There are some problems in dataset :
1. In location column there are some samples are like this ```"682 688;695 697"``` , which should be like ```"682 697"```
2. In which samples location is like above In that sample annotation is not proper example: ```"Father MI"``` and in text it is actually like this ```"Father had a MI"```
3. In some sample annotations are merging in each other examples:
    * 1st annotation's location is ```"331 368"``` and 2nd annotation's location ```"337 371"```
    * 1st annotation's location is ```"331 435"``` and 2nd annotation's location ```"337 371"```
4. In some sample annotations are unrelated to feature_text
5. In some sample annotaions are too big like a sentence which making them unrelated to feature_text and annotations len must be from 1 to about 5-6 words 

In [ ]:
def preprocess_location(text,as_type):
    """
    text: a string contain location eg. "333 378", "333 342;353 378", "5 9;15 35"
    as_type: int or str
    return: a string contain location eg. "333 378", "333 378", "5 35" as str or int 
    """
    text = re.findall(r"\d+",text)
    if as_type=="int":
        return int(text[0]), int(text[-1])
    elif as_type=="str":
        
        return text[0] + " " + text[-1]

def problem_process(df,location,annotation):
    """
    This function takes datafarame as input and solves avove problem 1 and 2 and return new dataset
    """
    
    
    pn_history = df.pn_history.values
    
    for i in range(len(annotation)):
        annotation[i] = eval(annotation[i]) # to convert ["[]","[]"] to [[],[]]
        location[i] = eval(location[i])
    
    new_location = copy.deepcopy(location)
    new_annotation = copy.deepcopy(annotation)
    
    """
    errors = []
    for i in range(len(location)):
        if location[i] != []:
            for j in range(len(location[i])):
                if len(location[i][j])>7:
                    errors.append([i,j])
    """
    
    trouble = [] # to collect sample who has problem no.3 
    
    for i in range(len(location)):
        if location[i]!=[]:
            sample = []
            for j in range(len(location[i])):
                text = location[i][j]
                new_location[i][j] = preprocess_location(text=text, as_type="str") # solving problem 1
                
                start, end = preprocess_location(text=text,as_type="int")
                new_annotation[i][j] = pn_history[i][start:end]                   # solving problem 2
                
                sample.append(start)
                sample.append(end)
            for m,n in enumerate(sample[1:]):
                if sample[m]>n:
                    trouble.append(i)
                    break

    df["annotation"] = np.array(new_annotation)
    df["location"] = np.array(new_location)
    
    
    return df, trouble

In [ ]:
train_df,trouble = problem_process(train_df,train_df.location.values,train_df.annotation.values)
incorrect = [0] * len(train_df.location.values)
for i in trouble:
    incorrect[i] = 1
train_df["trouble"] = np.array(incorrect)
#df,trouble = problem_process(df)
print(train_df.shape)
train_df = train_df[train_df["trouble"] == 0].reset_index(drop=True) # for now i dont want to solve problem 3 so i will just drop those rows

<div class="list-group" id="list-tab" role="tablist">
<h2 class="list-group-item list-group-item-action active" data-toggle="list" style='background:orange; border:0; color:white' role="tab" aria-controls="home"><center>Preprocessing</center></h2>

After applying preprocessing on pn_history location of annotations changes so using ```apply_preprocess_and_find_location```  we are applying preprocesing and calculating new location.
You can play with ```process_pn_history```, ```process_annotation```, ```process_feature_text``` function and apply different preprocessing whatever you like

In [ ]:
def process_pn_history(text):
    text = re.sub(r"\r\n"," ",text)
    return text


def process_annotation(text):
    text = re.sub(r"\r\n"," ",text)
    text = re.sub(r"\r","",text)
    return text

def process_feature_text(text):
    text = re.sub(r"-"," ",text)
    return text


def apply_preprocess_and_find_location(df):
    df["pn_history"] = df.pn_history.apply(process_pn_history)
    df["feature_text"] = df.feature_text.apply(process_feature_text)

    location = df.location.values
    annotation = df.annotation.values
    
    new_location = []
    pn_history = df.pn_history.values
    
    for i in range(len(df.annotation.values)):
        if annotation[i]==[]:
            new_location.append([])
        else:
            sample = []
            start = 0
            for j in range(len(annotation[i])):
                text = process_annotation(annotation[i][j]) # applied preprocess for annotation too
                start = pn_history[i].index(text, start, len(pn_history[i]))
                end = start + len(text)
                ii = str(start) + " " + str(end)
                sample.append(ii)
                start = end
            new_location.append(sample)
    
    
    df["location"] = np.array(new_location)
    return df

train_df = apply_preprocess_and_find_location(train_df)
train_df = train_df.drop([0,98,197,295,394],axis=0).reset_index(drop=True) # 0,98,197,295,394 are contain ids which mentioned in test_df

If you dont know about what is attention, transformers, tokenizer, bert. Then go take look at Hugging face's course. It is super easy and so much helpfull.

<div class="list-group" id="list-tab" role="tablist">
<h2 class="list-group-item list-group-item-action active" data-toggle="list" style='background:orange; border:0; color:white' role="tab" aria-controls="home"><center>Config</center></h2>

In [ ]:
class args:
    MAX_LEN = 300
    MODEL_PATH = "../input/bertbaseuncased/"
    MODEL_SAVE_PATH = "model.bin"
    n_splits = 10
    tokenizer = tokenizers.BertWordPieceTokenizer("../input/bertbaseuncased/vocab.txt",lowercase=True)
    seed = 42
    patiens = 5
    epochs = 2
    threshold=0.5
    learning_rate = 1e-4
    device="cuda"
    TRAIN_BATCH_SIZE = 16
    VALID_BATCH_SIZE = 16
    TEST_BATCH_SIZE = 5


<div class="list-group" id="list-tab" role="tablist">
<h2 class="list-group-item list-group-item-action active" data-toggle="list" style='background:orange; border:0; color:white' role="tab" aria-controls="home"><center>Utils</center></h2>

In [ ]:
class AverageMeter:
    """Computes and stores the average and current value"""
    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count
        

# setting seed so i can reproduce same results every time 
def set_seed(seed=args.seed):
    np.random.seed(seed)
    random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True
    os.environ["PYTHONHASHSEED"] = str(seed)
    

def confusion_metrics(ground_truth, prediction):
    ground_truth_elements = set(ground_truth)
    prediction_elements = set(prediction)
    
    TP = len(ground_truth_elements & prediction_elements)
    FN = len(ground_truth_elements - prediction_elements)
    FP = len(prediction_elements - ground_truth_elements)
    
    return TP, FN, FP

def micro_average_f1(TP, FN, FP):
    tp = sum(TP)
    fn = sum(FN)
    fp = sum(FP)
    f1 = tp / (tp + 0.5*(fp + fn))
    return f1

<div class="list-group" id="list-tab" role="tablist">
<h2 class="list-group-item list-group-item-action active" data-toggle="list" style='background:orange; border:0; color:white' role="tab" aria-controls="home"><center>Dataset</center></h2>

In [ ]:
class NBMEDataset:
    def __init__(self,df,tokenizer):
        self.df = df
        self.pn_history = df.pn_history.values
        self.feature_text = df.feature_text.values
        self.annotation = df.annotation.values
        self.location = df.location.values
        self.max_len = args.MAX_LEN
        self.tokenizer = tokenizer
    
    def __len__(self):
        return self.df.shape[0]
    
    def __getitem__(self,item):
        text = self.pn_history[item]
        feature_text = self.feature_text[item]
        annotation = self.annotation[item]
        location = self.location[item]
        
        
        len_featutre_text = len(feature_text)
        len_text = len(text)
        """
        lets say we have text(pn_history) like this:-  'This is first sentence' and annotaion is 'first sentence'
        then we will assign 0 to all and 1 to that index which contain our annotation except " " 
        example:
        "This is first sentence"
         0000000011111011111111   >> This will be in list format like [0,0,0,0,0,0,0,0,1,1,1,1,1,0,1,1,1,1,1,1,1,1]
        """
        char_targets = [0] * len_text
        if annotation!=[]:
            
            for idx0, idx1 in [(int(j) for j in i.split()) for i in location]:
                for j in range(idx0, idx1):
                    if text[j] != " ":
                        char_targets[j] = 1
        """
        Then we will give our text and feature_text to tokenizer and tokens will be like
        example : text >>> "I completed huggingface's course"
                tokens >>> ["i","completed","hugging","##face","'","s","course"]
        """
        
        tok_text = self.tokenizer.encode(text,feature_text)
        tok_text_tokens = tok_text.tokens
        tok_text_ids = tok_text.ids
        tok_text_type_ids = tok_text.type_ids
        tok_text_offsets = tok_text.offsets[1:-1]
        
        """
        Then we will create our targats 
        offsets are just starting and ending index of tokens like this [(0,0),(1,5),(6,9),(10,15),(15,19)]
        we will take offsets of each token and if that offsets sum(char_targets[start:end])>0 then that token is our target
        For example 
         annotation >>> "first sentence"
               text >>> "This is first sentence"
             tokens >>> ["this","is","first","sentence"]                    # I know there is [CLS] and [SEP] token at start and 
             offset >>> [(0,4),(5,7),(8,13),(14,22)]                        # end but i removed first and last offset
        char_target >>> [0,0,0,0,0,0,0,0,1,1,1,1,1,0,1,1,1,1,1,1,1,1]       # and also did -2 in len of tokens while creating target
             target >>> [0,0,1,1]
        
        """
        targets = [0] * (len(tok_text_tokens) - 2)
        for j, (offset1, offset2) in enumerate(tok_text_offsets):
            if sum(char_targets[offset1:offset2]) > 0:
                targets[j] = 1
        
        
        targets = [0] + targets + [0]                   # for cls and sep token
        
        
        mask = [1] * len(tok_text_ids)
        token_type_ids = tok_text_type_ids
        padding_len = self.max_len - len(tok_text_ids)
        
        ids = tok_text_ids + [0] * padding_len
        mask = mask + [0] * padding_len
        token_type_ids = token_type_ids + [0] * padding_len
        targets = targets + [0] * padding_len
        offsets = tok_text.offsets + [(0,0)] * padding_len
        
    
        
        return {
            "ids":torch.tensor(ids,dtype=torch.long),
            "mask":torch.tensor(mask,dtype=torch.long),
            "token_type_ids":torch.tensor(token_type_ids,dtype=torch.long),
            "targets":torch.tensor(targets,dtype=torch.float),
            "offsets":str(offsets),
            "location":str(location)
            
        }

<div class="list-group" id="list-tab" role="tablist">
<h2 class="list-group-item list-group-item-action active" data-toggle="list" style='background:orange; border:0; color:white' role="tab" aria-controls="home"><center>Model</center></h2>

In [ ]:
class BERTBaseUncased(nn.Module):
    def __init__(self):
        super(BERTBaseUncased, self).__init__()
        self.bert = transformers.BertModel.from_pretrained(args.MODEL_PATH)
        self.bert_drop = nn.Dropout(0.3)
        self.l0 = nn.Linear(768,1)
        
    def forward(self, ids, mask, token_type_ids):
        output = self.bert(
            ids,
            attention_mask=mask,
            token_type_ids=token_type_ids,
        )
        
        
        logits = self.l0(output[0])
        logits = logits.squeeze(-1)
        
        return logits

<div class="list-group" id="list-tab" role="tablist">
<h2 class="list-group-item list-group-item-action active" data-toggle="list" style='background:orange; border:0; color:white' role="tab" aria-controls="home"><center>Engine</center></h2>

In [ ]:
def loss_fn(o1,t1):
    l1 = nn.BCEWithLogitsLoss()(o1, t1)
    return l1 

def train_fn(dataloader, model, optimizer, device, scheduler):
    model.train()
    losses = AverageMeter()
    tk0 = tqdm(dataloader, total=len(dataloader))
    for bi, d in enumerate(tk0):
        ids = d["ids"]
        token_type_ids = d["token_type_ids"]
        mask = d["mask"]
        targets = d["targets"]
        
        ids = ids.to(device)
        token_type_ids = token_type_ids.to(device)
        mask = mask.to(device)
        targets = targets.to(device)
        
        optimizer.zero_grad()
        o1 = model(
            ids=ids,
            token_type_ids=token_type_ids,
            mask=mask
        )
        
        loss = loss_fn(o1,  targets)
        
        loss.backward()
        optimizer.step()
        scheduler.step()
        
        losses.update(loss.item(), ids.size(0))
        tk0.set_postfix(loss=losses.avg)
        
        ids = ids.cpu().detach().numpy()
        token_type_ids = token_type_ids.cpu().detach().numpy()
        mask = mask.cpu().detach().numpy()
        targets = targets.cpu().detach().numpy()
        del ids
        del token_type_ids
        del mask
        del targets
        del o1
    return losses.avg
          

def eval_fn(dataloader, model, device):
    model.eval()
    losses = AverageMeter()
    tk0 = tqdm(dataloader, total=len(dataloader))
    fin_output = []
    fin_targets = []
    fin_offsets = []
    orig_targets_location = []
    fin_targets_location = []
    fin_output_location = []
    TP = []
    FN = []
    FP = []
    
    for bi, d in enumerate(tk0):
        ids = d["ids"]
        token_type_ids = d["token_type_ids"]
        mask = d["mask"]
        targets = d["targets"]
        offsets = d["offsets"]
        location = d["location"]
        
        
        for i in range(len(location)):
            location[i] = eval(location[i])
            offsets[i] = eval(offsets[i])
        fin_offsets.extend(offsets)
        orig_targets_location.extend(location)
        
        
        ids = ids.to(device)
        token_type_ids = token_type_ids.to(device)
        mask = mask.to(device)
        targets = targets.to(device)
        
        
        o1 = model(
            ids=ids,
            token_type_ids=token_type_ids,
            mask=mask
        )
        
        loss = loss_fn(o1, targets)
        
        
        losses.update(loss.item(), ids.size(0))
        tk0.set_postfix(loss=losses.avg)
        
        ids = ids.cpu().detach().numpy()
        token_type_ids = token_type_ids.cpu().detach().numpy()
        mask = mask.cpu().detach().numpy()
        del ids
        del token_type_ids
        del mask
        
        threshold = args.threshold
        
        fin_output.append(torch.sigmoid(o1).cpu().detach().numpy())
        fin_targets.append(targets.cpu().detach().numpy())
        del o1
        del targets
        
        
    fin_offsets = np.array(fin_offsets)
    orig_targets_location = np.array(orig_targets_location)
    
    fin_output = np.vstack(fin_output)
    fin_targets = np.vstack(fin_targets)
    
    
    for i in range(len(fin_output)):
        output = [1 if i>=threshold else 0 for i in fin_output[i]]
        target = fin_targets[i]
        offset = fin_offsets[i]
        output_location = []
        start = -1
        for j in range(len(output)):
            if output[j]==1 and start==-1:
                start = offset[j][0]
            if output[j]==0 and start!=-1:
                end = offset[j-1][-1]
                output_location.extend([ii for ii in range(start,end)])
                start=-1
            
        fin_output_location.append(output_location)
    
    
    for i in orig_targets_location:
        orig_location = []
        for j in  i:
            m = j.split()
            start = int(m[0])
            end = int(m[-1])
            orig_location.extend([ii for ii in range(start,end)])
        fin_targets_location.append(orig_location)

    for i in range(len(orig_targets_location)):
        ground_truth = fin_targets_location[i]
        prediction = fin_output_location[i]
        
        tp, fn, fp = confusion_metrics(ground_truth,prediction)
        TP.append(tp)
        FN.append(fn)
        FP.append(fp)
    
    micro_average_f1_score = micro_average_f1(TP, FN, FP)
        
    return losses.avg, micro_average_f1_score

<div class="list-group" id="list-tab" role="tablist">
<h2 class="list-group-item list-group-item-action active" data-toggle="list" style='background:orange; border:0; color:white' role="tab" aria-controls="home"><center>Train</center></h2>

In [ ]:
train_df["kfold"] = -1
set_seed(args.seed)
skf = model_selection.StratifiedKFold(n_splits=args.n_splits,shuffle=True,random_state=args.seed)
y=train_df.case_num.values
for fold_, (_,v_) in enumerate(skf.split(X=train_df,y=y)):
    train_df.loc[v_,"kfold"] = fold_

In [ ]:
valid_score = []
fold_no = []
for fold in range(args.n_splits):
    set_seed(args.seed + fold)
    train_data = train_df[train_df.kfold!=fold].reset_index(drop=True)
    valid_data = train_df[train_df.kfold==fold].reset_index(drop=True)
    
    train_dataset = NBMEDataset(df=train_data,tokenizer=args.tokenizer)
    valid_dataset = NBMEDataset(df=valid_data,tokenizer=args.tokenizer)

    set_seed(args.seed+fold)
    
    train_dataloader = DataLoader(
        train_dataset,
        batch_size=args.TRAIN_BATCH_SIZE,
        num_workers=2
    )
    
    set_seed(args.seed+fold)
    
    valid_dataloader = DataLoader(
        valid_dataset,
        batch_size=args.VALID_BATCH_SIZE,
        num_workers=2
    )
    
    set_seed(args.seed+fold)
    
    model = BERTBaseUncased()
    model.to(args.device)
    
    param_optimizer = list(model.named_parameters())
    no_decay = ["bias","LayerNorm.bias","LayerNorm.weight"]
    optimizer_parameters = [
        {"params":[p for n,p in param_optimizer if not any(nd in n for nd in no_decay)],"weight_decay":0.001}
    ]
    
    num_train_steps = int(len(train_df) / args.TRAIN_BATCH_SIZE * args.epochs)
    optimizer = AdamW(optimizer_parameters, lr=args.learning_rate)
    scheduler = get_linear_schedule_with_warmup(
        optimizer,
        num_warmup_steps=0,
        num_training_steps=num_train_steps
    )
    
    
    
    best_micro_averaged_F1 = 0
    patiens = 0
    for epoch in range(args.epochs):
        print(f"fold/epoch  {fold}/{epoch}")
        
        tr_loss = train_fn(train_dataloader, model, optimizer, args.device, scheduler)
        gc.collect()
        torch.cuda.empty_cache()
        
        val_loss, micro_average_f1_score = eval_fn(valid_dataloader, model, args.device)
        gc.collect()
        torch.cuda.empty_cache()
        
        
        patiens+=1
        print(f"valid_score = {micro_average_f1_score}")
        
        if micro_average_f1_score>=best_micro_averaged_F1:
            best_micro_averaged_F1 = micro_average_f1_score
            torch.save(model.state_dict(),args.MODEL_SAVE_PATH)
            patiens=0
        
        if patiens==args.patiens:
            break
        
    valid_score.append(best_micro_averaged_F1)
    fold_no.append(fold)
    print("*"*30)
    break # remove break to train on all folds

I used only 2 epochs here to just show you that its works, you should used 20-25 epoch. Yes I know i not explained  about how to get pred location from predicted target i will leave that to you
     
     hint >>> used offset to get location  >> its inside eval_fn

<font color='red'>If you find my kernel useful and my efforts appreciable, Please Upvote it , it motivates me to write more Quality content</font>

<div class="list-group" id="list-tab" role="tablist">
<h2 class="list-group-item list-group-item-action active" data-toggle="list" style='background:orange; border:0; color:white' role="tab" aria-controls="home"><center>End</center></h2>